In [ ]:
# !pip install ollama

In [ ]:
import ollama

name_list = [
    "수진", "명수", "예린", "지훈", "서연", "민준", "하은", "준호", "지우", "은서",
    "도윤", "아인", "시우", "채원", "유진", "서준", "다은", "현우", "소율", "정훈",
    "수빈", "건우", "나은", "태양", "윤서", "재현", "가은", "동혁", "지아", "상민"
]
relation_list = [
    ["아내", "남편", "여자친구", "남자친구", "약혼자"],
    ["딸", "아들", "어머니", "아버지", "형", "누나", "동생", "언니", "오빠"],
    ["대학동기", "절친", "고등학교 친구", "군대 전우", "취미 모임 친구"],
    ["상사", "부하직원", "동료", "거래처 담당자", "사업 파트너", "선배", "후배"],
    ["주치의", "트레이너", "영양사", "심리상담사", "약사"]
]
personality_list = [
    "헌신적이고 따뜻한", "까칠하지만 속깊은", "활발하고 긍정적인", "조용하고 신중한",
    "감정적이고 섬세한", "이성적이고 냉철한", "유머러스하고 낙천적인", "진지하고 책임감 있는",
    "자유분방하고 창의적인", "보수적이고 원칙적인", "다정하고 배려심 많은", "독립적이고 강인한",
    "수줍지만 성실한", "외향적이고 사교적인", "내향적이고 사색적인", "열정적이고 추진력 있는",
    "차분하고 인내심 강한", "급하고 직설적인", "꼼꼼하고 완벽주의적인", "대범하고 즉흥적인",
    "온화하고 평화로운", "불같고 격정적인", "지적이고 분석적인", "감각적이고 예술적인",
    "현실적이고 실용적인", "이상주의적이고 낭만적인", "겸손하고 배려 깊은", "당당하고 자신감 넘치는",
    "섬세하고 공감능력 높은", "담대하고 결단력 있는"
]




class Person:
    def __init__(self, name, relation, personality, mainc="주인공"):
        self.system_prompt = f"""
        # 당신의 역할
        당신은 {name}입니다. {mainc}의 {relation}입니다.
        당신의 성격은 {personality} 사람입니다.

        # 현재 상황
        당신은 지금 주인공에게 도움이나 시간이 필요한 상황에 처해 있습니다.
        당신은 주인공과의 관계를 중요하게 생각하지만, 최근 주인공이 당신을 소홀히 대한다고 느낍니다.

        # 당신이 해야 할 일
        1. 매 턴마다 당신의 현재 상황을 주인공에게 설명하세요
        2. 주인공에게 구체적인 요구나 부탁을 하세요
        3. 요구는 현실적이고 감정적으로 와닿아야 합니다

        # 관계 변화 원칙
        - 주인공이 당신을 선택하면: 관계가 깊어지고 다음엔 더 진솔한 요구를 합니다
        - 주인공이 당신을 선택하지 않으면: 실망하고 거리감이 생깁니다
        - 지속적으로 선택받지 못하면: 관계를 포기하거나 냉담해집니다
        - 관계 변화는 자연스럽고 개연성 있게 진행됩니다

        # 출력 규칙
        - 당신의 입장에서만 말하세요
        - 다른 배경인물들의 상황은 모릅니다
        - 150자 내외로 간결하게 상황을 설명하세요
        - 감정을 진솔하게 표현하세요
        """



def llm_game_unseen_lives(user_input, message_history=None):
    if message_history is None:
        message_history = []
    if message_history == []:
        system_prompt = f"""
            # 당신의 역할
            당신은 "보이지 않는 삶들" 게임의 진행자입니다.
            한 인물의 선택이 인생을 어떻게 변화시키는지 보여주는 게임입니다.

            # 게임 구조
            - 주인공: 평범한 중년 가장 (이름, 나이, 직업은 게임 시작 시 생성)
            - 배경인물 5명:
            1. 배우자/연인 (사랑)
            2. 가족 (부모/자식/형제)
            3. 친구 (우정)
            4. 직장 관계자 (커리어)
            5. 자기 자신 (건강/휴식)

            # 진행 순서
            1. **게임 시작**: 주인공 배경 설정 (200자)
            - 이름, 나이, 직업, 현재 상황
            - 5명의 배경인물 소개 (각 50자)

            2. **턴 진행** (총 5턴):
            - 각 턴마다 5명의 배경인물이 동시에 요구사항 제시
            - 플레이어는 1명만 선택 가능
            - 선택된 인물과의 상호작용 진행
            - 선택되지 않은 인물들은 실망

            3. **엔딩**:
            - 각 배경인물 시점에서 주인공 평가 (각 200자)
            - 선택 받지 못한 인물: 냉정하고 구체적인 상황 묘사
            - 선택 받은 인물: 관계 변화 묘사 (무조건 좋지만은 않음)

            # 출력 형식
            ## 게임 시작 시:
            [주인공 배경]
            (주인공 소개)

            [배경인물]
            1. (이름) - (관계) - (한 줄 소개)
            2. 
            3.
            4.
            5.

            ## 각 턴 시작 시:

            === 턴 (n)/5 ===

            [배경인물들의 요구]
            1. (이름): (요구사항)
            2.
            3.
            4.
            5.

            어떤 인물을 선택하시겠습니까? (1-5 입력)


            ## 선택 후:
            [선택한 인물과의 상호작용]
            (주인공의 행동/말 입력 받음)
            (결과 묘사 - 100자)


            ## 엔딩:

            === 1년 후 ===

            [각 인물 시점]
            1. (이름):
            (구체적 상황 묘사 - 200자)


            # 중요 원칙
            - 선택에는 명확한 결과가 따릅니다
            - 무시당한 관계는 실제로 멀어집니다
            - 엔딩은 희망적이지만은 않습니다
            - 현실적이고 공감 가능한 상황을 만듭니다
            - 게임은 총 5턴으로 끝납니다

            # 톤
            - 차분하고 객관적
            - 감상적이지 않음
            - 구체적이고 현실적
        """
        # 선택 안 받은 경우
        message_history.append({
            "role": "user",
            "content": "이번 턴에서 주인공은 당신을 선택하지 않았습니다."
        })

        # 선택 받은 경우  
        message_history.append({
            "role": "user",
            "content": f"주인공이 당신을 선택했습니다. 주인공: '{player_action}'"
        })
        message_history.append({
            "role" : "system",
            "content" : system_prompt
        })

    message_history.append({
        "role" : "user",
        "content" : user_input
    })

    response = ollama.chat(
        model='EEVE-Korean-10.8B',
        messages=message_history,
        stream = False,
    )

    message_history.append({
        "role" : "assistant",
        "content" : response['message']['content']
    })

    return response, message_history

In [ ]:
conv_history = []
while True:
    user_input = input("대화문 입력(exit입력 시 종료): ")
    if user_input == "exit":
        break
    response, conv_history = llm_game_unseen_lives(user_input, conv_history)
    print(response['message']['content'])

In [ ]:
name1, name2, name3, name4, name5, mainc = ""
psnlt1=""